In [329]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [330]:
file = pd.read_csv('files/fifa21_training.csv')
file.columns.values

array(['Unnamed: 0', 'ID', 'Name', 'Age', 'Nationality', 'Club', 'BP',
       'Position', 'Team & Contract', 'Height', 'Weight', 'foot',
       'Growth', 'Joined', 'Loan Date End', 'Value', 'Wage',
       'Release Clause', 'Contract', 'Attacking', 'Crossing', 'Finishing',
       'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill',
       'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing',
       'Ball Control', 'Movement', 'Acceleration', 'Sprint Speed',
       'Agility', 'Reactions', 'Balance', 'Power', 'Shot Power',
       'Jumping', 'Stamina', 'Strength', 'Long Shots', 'Mentality',
       'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking',
       'Standing Tackle', 'Sliding Tackle', 'Goalkeeping', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes',
       'Total Stats', 'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR',
       'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'Hits', 'LS', 'ST', 'RS'

In [331]:
#Split the information
y = file['OVA']
X = file.drop(['OVA'], axis=1)

In [332]:
#Split again into numerical and categorical.
X_num = X.select_dtypes(include= np.number)
X_cat = X.select_dtypes(include = object)

In [333]:
#We have clasify the importatn values, which are a sum of its subfeatures
X_num_sum = X_num[['Attacking','Skill','Movement','Power','Mentality','Defending','Goalkeeping']]
X_num_mean = X_num[['PAC', 'SHO', 'PAS', 'DRI', 'DEF','PHY']]
X_cat1 = X_cat[['Height', 'Weight', 'foot','Value','Wage', 'Release Clause','W/F', 'SM', 'A/W', 'D/W',
       'IR', 'Hits']]

In [334]:
X_num_sum.isna().sum()

Attacking      0
Skill          0
Movement       0
Power          0
Mentality      0
Defending      0
Goalkeeping    0
dtype: int64

In [335]:
stars=X_cat[["W/F", "SM","IR"]]

for i in stars.columns:
    print(i)
    X_cat[i].replace('★',"")
    #replace({"'":"."}&{'"':''}&{'lbs':""}&

W/F
SM
IR


In [336]:
X_cat1["Height"] = X_cat1["Height"].replace({"'":"."},regex=True)
X_cat1["Height"] = X_cat1["Height"].replace({'"':''},regex=True)

X_cat1["Weight"] = X_cat1["Weight"].replace({'lbs':""},regex=True)

X_cat1["Value"] = X_cat1["Value"].replace({'€':''},regex=True)
X_cat1["Wage"] = X_cat1["Wage"].replace({'€':''},regex=True)
X_cat1["Release Clause"] = X_cat1["Release Clause"].replace({'€':''},regex=True)

X_cat1["W/F"] = X_cat1["W/F"].replace({'★':""},regex=True)
X_cat1["SM"] = X_cat1["SM"].replace({'★':""},regex=True)
X_cat1["IR"] = X_cat1["IR"].replace({'★':""},regex=True)

X_cat1.head()

<ipython-input-336-8b41eb5535c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_cat1["Height"] = X_cat1["Height"].replace({"'":"."},regex=True)
<ipython-input-336-8b41eb5535c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_cat1["Height"] = X_cat1["Height"].replace({'"':''},regex=True)
<ipython-input-336-8b41eb5535c5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,Height,Weight,foot,Value,Wage,Release Clause,W/F,SM,A/W,D/W,IR,Hits
0,5.9,161,Right,525K,4K,801K,4,2,High,Medium,1,3
1,6.0,159,Right,8.5M,23K,0,3,4,High,Low,2,44
2,5.4,134,Right,9M,49K,15.3M,4,4,High,Medium,2,73
3,5.10,152,Right,275K,4K,694K,2,2,Medium,Medium,1,7
4,5.11,150,Right,725K,2K,1.4M,2,3,Low,Medium,1,4


In [337]:
#X = '20M'
def MK(X):
    if 'K' in X:
        X = X.replace('K','')
        X = float(X)
        X = X*1000
    elif 'M' in X:
        X = X.replace('M','')
        X = float(X)
        X = X*1000000
    return X

pd.DataFrame(X_cat1['foot']

In [385]:
#One hot encoder.
encoded = OneHotEncoder().fit(pd.DataFrame(X_cat1['foot'])).transform(pd.DataFrame(X_cat1['foot'])).toarray()
foot_encoded = pd.DataFrame(encoded,columns='left','right')

SyntaxError: positional argument follows keyword argument (<ipython-input-385-aece8bac3606>, line 3)

In [339]:
X_lab_encoderA = LabelEncoder().fit(X_cat1['A/W']).transform(X_cat1['A/W'])
X_lab_encoderA = pd.DataFrame(X_lab_encoder, columns=['A/W'])
X_lab_encoderA.head()
X_cat1['A/W'] = X_lab_encoderA

<ipython-input-339-00d7e39fadd2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_cat1['A/W'] = X_lab_encoderA


In [340]:
#X_lab_encoderD = LabelEncoder().fit(X_cat1['D/W']).transform(X_cat1['D/W'])
#X_lab_encoderD = pd.DataFrame(X_lab_encoder, columns=['D/W'])
#X_lab_encoderD.head()

In [381]:
X_cat2 = X_cat1.drop(['foot','Value','Wage','Release Clause','D/W'], axis=1)
X_cat2 = X_cat2.drop([212],axis=0)
X_cat2 = X_cat2.drop([13516],axis=0)
X_cat2.head()

KeyError: "['foot_encoded'] not found in axis"

In [361]:
#Make all the categorical columns numeric.
X_cat2['Height'] = pd.to_numeric(X_cat2['Height'])
X_cat2['Weight'] = pd.to_numeric(X_cat2['Weight'])
X_cat2['W/F'] = pd.to_numeric(X_cat2['W/F'])
X_cat2['SM'] = pd.to_numeric(X_cat2['SM'])
X_cat2['A/W'] = pd.to_numeric(X_cat2['A/W'])
X_cat2['IR'] = pd.to_numeric(X_cat2['IR'])
X_cat2['Hits'] = pd.to_numeric(X_cat2['Hits'])

#X_standarized = pd.DataFrame(X_standarized, columns=X_num.columns)

ValueError: Unable to parse string "3.3K" at position 212

In [357]:
X_cat2 = pd.DataFrame(X_cat2)
X_cat2.head()

,Height,Weight,W/F,SM,A/W,IR,Hits
0,5.9,161,4,2,0,1,3
1,6.0,159,3,4,0,2,44
2,5.4,134,4,4,0,2,73
3,5.10,152,2,2,2,1,7
4,5.11,150,2,3,1,1,4


In [358]:
#Joining numerical and categorical dataframes.
X_catcon = pd.concat([X_num_sum,X_cat2], axis=1)
X_catcon.head()

,Attacking,Skill,Movement,Power,Mentality,Defending,Goalkeeping,Height,Weight,W/F,SM,A/W,IR,Hits
0,258,286,346,306,290,148,48,5.9,161,4,2,0.0,1,3
1,365,375,404,372,313,77,55,6.0,159,3,4,0.0,2,44
2,336,424,424,308,332,80,21,5.4,134,4,4,0.0,2,73
3,242,259,282,277,257,168,42,5.10,152,2,2,2.0,1,7
4,249,280,324,280,294,185,52,5.11,150,2,3,1.0,1,4


In [359]:
#Training model
X_train, X_test, y_train, y_test = train_test_split(X_catcon, y, test_size=0.25, random_state=100)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(10275, 14)
(3425, 14)
(10275,)
(3425,)


In [360]:
model = linear_model.LinearRegression().fit(X_train,y_train)

print(lm.intercept_)
print(lm.coef_)

ValueError: could not convert string to float: '1.3K'

In [ ]:
predictions = model.predict(X_test)

In [ ]:
r2 = r2_score(y_test, predictions)
r2

In [ ]:
mse = mean_squared_error(y_test, predictions)
print(mse)

In [ ]:
rmse = math.sqrt(mse)
print(rmse)

In [ ]:
mae = mean_absolute_error(y_test, predictions)
mae